In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
from tqdm import tqdm
import os


In [2]:
# ======================
# CONFIG
# ======================
DATA_DIR = "cnn_dataset"
IMG_SIZE = 224
BATCH_SIZE = 32
# EPOCHS = 20
EPOCHS = 30
# LR = 1e-4
LR = 3e-5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

SAVE_PATH = "efficientnetv2s_best_finetuned_best.pth"

In [3]:
# ======================
# TRANSFORMS
# ======================
train_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ColorJitter(brightness=0.2, contrast=0.2),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

val_tfms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

In [4]:
# ======================
# DATASETS
# ======================
train_ds = datasets.ImageFolder(f"{DATA_DIR}/train", train_tfms)
val_ds   = datasets.ImageFolder(f"{DATA_DIR}/val", val_tfms)

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=4)
val_loader   = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=4)

print("Classes:", train_ds.classes)

NUM_CLASSES = len(train_ds.classes)

Classes: ['1509', 'IRRI-6', 'Super White']


In [5]:
# ======================
# MODEL (EfficientNetV2-S)
# ======================
model = models.efficientnet_v2_s(weights="DEFAULT")

# unfreeze all
for param in model.parameters():
    param.requires_grad = True
# replace classifier
in_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(in_features, NUM_CLASSES)

model = model.to(DEVICE)

In [6]:
# ======================
# LOSS / OPTIMIZER
# ======================
criterion = nn.CrossEntropyLoss(label_smoothing=0.05)
# optimizer = optim.Adam(model.parameters(), lr=LR)
optimizer = torch.optim.AdamW(model.parameters(), lr=LR, weight_decay=1e-4)

In [7]:
# ======================
# TRAIN LOOP
# ======================
best_acc = 0

for epoch in range(EPOCHS):

    # ===== TRAIN =====
    model.train()
    correct, total = 0, 0

    for imgs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS}"):

        imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)

        loss.backward()
        optimizer.step()

        _, preds = outputs.max(1)
        correct += (preds == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total

    # ===== VALIDATION =====
    model.eval()
    correct, total = 0, 0

    with torch.no_grad():
        for imgs, labels in val_loader:

            imgs, labels = imgs.to(DEVICE), labels.to(DEVICE)
            outputs = model(imgs)

            _, preds = outputs.max(1)
            correct += (preds == labels).sum().item()
            total += labels.size(0)

    val_acc = correct / total

    print(f"\nTrain Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")

    if val_acc > best_acc:
        best_acc = val_acc
        torch.save(model.state_dict(), SAVE_PATH)
        print("✅ Best model saved")

print("\n🔥 Training finished")

Epoch 1/30: 100%|██████████| 282/282 [02:13<00:00,  2.11it/s]



Train Acc: 0.8123 | Val Acc: 0.9122
✅ Best model saved


Epoch 2/30: 100%|██████████| 282/282 [02:14<00:00,  2.10it/s]



Train Acc: 0.9383 | Val Acc: 0.9489
✅ Best model saved


Epoch 3/30: 100%|██████████| 282/282 [02:14<00:00,  2.10it/s]


...


Train Acc: 0.9961 | Val Acc: 0.9822


Epoch 29/30: 100%|██████████| 282/282 [02:14<00:00,  2.10it/s]



Train Acc: 0.9969 | Val Acc: 0.9867


Epoch 30/30: 100%|██████████| 282/282 [02:14<00:00,  2.10it/s]



Train Acc: 0.9963 | Val Acc: 0.9800

🔥 Training finished
